In [ ]:
#script to download 2020 Census blocks state by state 
import urllib.request
import os

# Define the base URL for the 2010 Census Blocks dataset
base_url = "https://www2.census.gov/geo/tiger/TIGER2020/TABBLOCK20/"

# Get a list of all the state FIPS codes
state_fips = range(1, 69)

# Define the working directory for downloaded files
working_directory = "C:/Users/ewiggans/OneDrive - Lincoln Institute of Land Policy/Desktop/GeoConnexMap/data/2020Census"
# Create the working directory if it does not exist
if not os.path.exists(working_directory):
    os.makedirs(working_directory)

# Loop through each state FIPS code and download the corresponding zip file
for fips in state_fips:
    # Construct the URL for the state's zip file
    url = base_url + "tl_2020_" + str(fips).zfill(2) + "_tabblock20.zip"
    # Construct the local file name for the downloaded zip file
    filename = os.path.join(working_directory, "tl_2020_" + str(fips).zfill(2) + "_tabblock20.zip")
    print(f"Downloading file from URL: {url}")
    # Download the zip file
    try:
        urllib.request.urlretrieve(url, filename)
        print("Downloaded state with FIPS code " + str(fips).zfill(2) + ".")
    except urllib.error.HTTPError as e:
        print(f"Failed to download state with FIPS code {str(fips).zfill(2)}: {e}")


In [ ]:
#unzip all files
import zipfile
import os

dir_path = "C:/Users/ewiggans/OneDrive - Lincoln Institute of Land Policy/Desktop/GeoConnexMap/data/2020Census"

# Loop through each file in the directory
for file_name in os.listdir(dir_path):
    # Check if the file is a zip file
    if file_name.endswith('.zip'):
        # Open the zip file
        with zipfile.ZipFile(os.path.join(dir_path, file_name), 'r') as zip_ref:
            # Extract all the files in the zip file to the same directory
            zip_ref.extractall(dir_path)
print("Unzipped files")

In [ ]:
##Script to intersect 2020 Census Blocks with NHDPlusV2 catchments
##2/14/2022

#Data Sources - 
##Using: https://www2.census.gov/geo/tiger/TIGER2020/TABBLOCK20/ as 2020 Census Blocks
##Catchments - Using: https://www.sciencebase.gov/catalog/item/61295190d34e40dd9c06bcd7

##Intersect Census Blocks for 2000, 2010, 2020 with NHDPlus V2 Catchments

##Need fields Census block year/id, 
#the catchment COMID, the % area of the census block that the intersection is, 
#and the % area of the catchment that the intersection is
import arcpy
import datetime
from arcpy import env
import time

# Enable overwrite output
arcpy.env.overwriteOutput = True

# Set the workspace to the folder where the census block data is held
env.workspace = r'C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2020Census'

# List all the shapefiles in the folder
shapefiles = arcpy.ListFeatureClasses(feature_type='Polygon')

# Print start time
start_time = datetime.datetime.now()
print(f'Start time: {start_time}')

# Add a field called "Area_sqkm" to each shapefile and calculate the area in square kilometers
for shapefile in shapefiles:
    arcpy.AddField_management(shapefile, "Area_sqkm", "DOUBLE")
    arcpy.CalculateGeometryAttributes_management(shapefile, [["Area_sqkm", "AREA_GEODESIC"]], "", "SQUARE_KILOMETERS")
    print(f'{shapefile} processed.')

# add delay of 5 seconds
time.sleep(5)

# Intersect each shapefile with the national NHD catchments dataset
catchment_dataset = r'C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\GeoConnex Map\GeoConnex Map.gdb\main_reference_catchments'
for shapefile in shapefiles:
    output_feature_class = f"inter_{shapefile}"
    arcpy.Intersect_analysis([shapefile, catchment_dataset], output_feature_class, "ALL", "", "INPUT")
    print(f'{output_feature_class} created.')

# add delay of 5 seconds
time.sleep(5)
    
# Print total time elapsed
end_time = datetime.datetime.now()
total_time = end_time - start_time
print(f'Total time elapsed: {total_time}')


In [ ]:
##For 2020 census data, we now have intersected files for census blocks and catchments
## The area for the catchments is field: AREASQKM
## The area for the census blocks is: Area_sqkm


import arcpy
import os
import time
import winsound

# Set the workspace to the directory containing the shapefiles
workspace = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2020Census"
arcpy.env.workspace = workspace

# Get a list of all shapefiles in the workspace that start with "inter"
shapefiles = [os.path.join(workspace, file) for file in os.listdir(workspace) if file.startswith("inter") and file.endswith(".shp")]

# Start the timer
start_time = time.time()

# Loop through each shapefile
for shapefile in shapefiles:
    print("Processing " + shapefile)
    
    # Add the "inter_sqkm" field
    arcpy.AddField_management(shapefile, "inter_sqkm", "DOUBLE")
    
    # Calculate the area of each feature in square kilometers and store it in the "inter_sqkm" field
    arcpy.CalculateGeometryAttributes_management(shapefile, [["inter_sqkm", "AREA_GEODESIC"]], area_unit="SQUARE_KILOMETERS")
    
    # Add the fields for calculating percent intersections
    arcpy.AddField_management(shapefile, "P_c_in_bk", "DOUBLE")
    arcpy.AddField_management(shapefile, "P_bk_in_c", "DOUBLE")
    
#Stop the timer
stop_time = time.time()

# Calculate the elapsed time
elapsed_time = stop_time - start_time
# Print a message with the elapsed time
print("completed successfully in " + str(round(elapsed_time, 2)) + " seconds.")

In [ ]:
#This code calculates the fields for percent block in catchment and percent catchment in block
import arcpy
import os
import time
# Set the workspace to the directory containing the shapefiles
workspace = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2020Census"
arcpy.env.workspace = workspace
# Start the timer
start_time = time.time()
# Get a list of all shapefiles in the workspace that start with "inter"
shapefiles = [os.path.join(workspace, file) for file in os.listdir(workspace) if file.startswith("inter") and file.endswith(".shp")]
# Loop through each shapefile
for shapefile in shapefiles:
    print("Processing " + shapefile)
    
    # Calculate the "P_c_in_bk" field ie the percent of the catchment in a block
    arcpy.AddField_management(shapefile, "P_c_in_bk", "DOUBLE")
    arcpy.CalculateField_management(shapefile, "P_c_in_bk", "(!inter_sqkm! / !AREASQKM!) * 100", "PYTHON3")
    
    # Calculate the "P_bk_in_c" field ie the percent of a block in a catchment 
    arcpy.AddField_management(shapefile, "P_bk_in_c", "DOUBLE")
    arcpy.CalculateField_management(shapefile, "P_bk_in_c", "(!inter_sqkm! / !Area_sqkm!) * 100", "PYTHON3")
# Stop the timer
stop_time = time.time()
# Calculate the elapsed time
elapsed_time = stop_time - start_time
# Print a message with the elapsed time
print("completed successfully in " + str(round(elapsed_time, 2)) + " seconds.")


In [ ]:
#This code merges together all the files into one feature class

import arcpy
import os
import time
import winsound

# Set the workspace to the directory containing the shapefiles
workspace = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2020Census"
arcpy.env.workspace = workspace

# Start the timer
start_time = time.time()
###################################################

# Set the workspace to the directory containing the shapefiles
workspace = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2020Census"
arcpy.env.workspace = workspace

# Get a list of all shapefiles in the workspace that start with "inter"
shapefiles = [os.path.join(workspace, file) for file in os.listdir(workspace) if file.startswith("inter") and file.endswith(".shp")]

# Merge all shapefiles that start with "inter"
merged_file = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\GeoConnex Map\GeoConnex Map.gdb\tl_2020_merge"
arcpy.Merge_management(shapefiles, merged_file)

###################################################
# Stop the timer
stop_time = time.time()

# Calculate the elapsed time
elapsed_time = stop_time - start_time

# Print a message with the elapsed time
print("completed successfully in " + str(round(elapsed_time, 2)) + " seconds.")


In [ ]:
##field clean up
import arcpy


#field clean up for centroids
input_fc = "tl_2020_merge_centroid"
arcpy.DeleteField_management(input_fc, ["FEATUREID_calc", "GEOID20_calc", "ORIG_FID"])

#field clean up for polygons
input_fc = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2020Census\2020Census.gdb\tl_2020_merge"

arcpy.DeleteField_management(input_fc, ["FEATUREID_calc", "GEOID_calc"])

In [ ]:
import arcpy
import os
import time

# Set the input folder and output geodatabase locations
input_folder = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2020Census"
output_gdb = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2020Census\2020Census.gdb"

# Set the merge output feature class name
merge_output_fc = "tl_2020_CONUS"

# Create an empty list to store the input feature classes
input_fc_list = []

# Iterate through the input folder and find all feature classes that start with "tl_"
for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.startswith("tl_") and file.endswith(".shp"):
            # Construct the full path to the feature class
            fc_path = os.path.join(root, file)
            # Add the feature class path to the list
            input_fc_list.append(fc_path)

# Use the arcpy Merge tool to merge the input feature classes into the output geodatabase
start_time = time.time()
arcpy.Merge_management(input_fc_list, os.path.join(output_gdb, merge_output_fc))
end_time = time.time()

# Print the number of input feature classes and the merge output feature class name
print("Merged {} input feature classes into feature class {}.".format(len(input_fc_list), merge_output_fc))

# Print the time it took to run the script
print("Script completed in {:.2f} seconds.".format(end_time - start_time))
#but may include non CONUS
print("If you do not want Alaska, Hawaii, etc. Delete")


In [ ]:
##Updated final code for updating uri/url, https://geoconnex.us/iow/nhdpv2-census/2020/comid 
###This will add the required fields, calculate them, rename as necessary, and remove unnecessary ones.
# # final steps to produce data
import arcpy
import winsound
import time
# Set the input and output feature classes


input_fc = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2020Census\2020Census.gdb\tl_2020_merge"
output_fc = r"C:\Users\ewiggans\GeoConnexC\GeoConnex C Drive\GeoConnex C Drive.gdb\USA_2020_merge_centroids"

# Start the timer
start_time = time.time()
print("started at")
print(start_time)


# Add new fields
arcpy.AddField_management(input_fc, "FEATUREID_calc", "TEXT")
arcpy.AddField_management(input_fc, "FEATUREID_int", "LONG")
arcpy.AddField_management(input_fc, "GEOID20_calc", "TEXT")

# Calculate new fields
arcpy.CalculateField_management(input_fc, "FEATUREID_int", "int(!FEATUREID!)", "PYTHON3")
arcpy.CalculateField_management(input_fc, "FEATUREID_calc", "str(!FEATUREID!)", "PYTHON3")
arcpy.CalculateField_management(input_fc, "FEATUREID_calc", "(!FEATUREID_calc!).rstrip('.0')", "PYTHON3")
arcpy.CalculateField_management(input_fc, "GEOID20_calc", "str(!GEOID20!)", "PYTHON3")

# Add the "uri", "id", and "name" fields
arcpy.AddField_management(input_fc, "uri", "TEXT")
arcpy.AddField_management(input_fc, "id", "TEXT")
arcpy.AddField_management(input_fc, "name", "TEXT")
print("Fields added")

# Calculate the "uri" field
feature_id = "(!FEATUREID_calc!)"
geoid20 = "(!GEOID20_calc!)"
base_url = '"https://geoconnex.us/iow/nhgf-census/2010/comid/"'
block_url = "/block/"

#thank you Ben, four for you Ben
codeblock = """
def url_join(*parts: list) -> str:
    return '/'.join([str(p).strip().strip('/') for p in parts])
"""

expression = """url_join("https://geoconnex.us/iow/nhdpv2-census/2020/comid", !FEATUREID_calc!, "block", !GEOID20_calc!)"""
arcpy.management.CalculateField(input_fc, "uri", expression, "PYTHON3", codeblock)
print("uri calculated")

# Calculate the "id" field
arcpy.management.CalculateField(input_fc, "id", """"comid_" + str(!FEATUREID_calc!) + "_block_" + str(!GEOID20_calc!)""", "PYTHON3")
print("id calculated")


# Calculate the "name" field
arcpy.CalculateField_management(input_fc, "name", "!id!", "PYTHON3")
print("name calculated")

#delete the _calc fields
arcpy.DeleteField_management(input_fc, ["FEATUREID", "FEATUREID_calc", "GEOID20_calc"])
print("fields deleted, now making centroids")

#rename the feature id field to the one that is an integer
arcpy.AlterField_management(input_fc, "FEATUREID_int", "FEATUREID")

#make a copy of the features on C drive
arcpy.management.CopyFeatures(input_fc, r"C:\Users\ewiggans\GeoConnexC\GeoConnex C Drive\GeoConnex C Drive.gdb\USA_2020_merge_final")

# Convert the polygon features to point centroids
arcpy.FeatureToPoint_management(input_fc, output_fc, "CENTROID")
print("You made centroids")

# Stop the timer
stop_time = time.time()

# Calculate the elapsed time
elapsed_time = stop_time - start_time
# Print a message for complete
print("completed successfully in " + str(round(elapsed_time, 2)) + " seconds.")
